1. Сканирование страниц, получение полного списка фильмов
2. Парсинг всех фильмов, сохранение комментариев и параметров в объектах набора данных
3. Создание индекса
4. Выделение комментариев в отдельный набор

In [24]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
URL_DORAMY = 'https://doramy.club/page/'

# Тестирование на скрабинг

In [25]:
r = requests.get(URL_DORAMY+str(1))

In [27]:
r.status_code

200

In [5]:
soup = BeautifulSoup(r.text)
page_films=soup.find_all("div", class_="post-home")
len(page_films)

In [16]:
film = page_films[0]

In [22]:
film

<div class="post-home">
<a href="https://doramy.club/34015-neozhidannaya-vstrecha.html"><img alt="" src="https://doramy.club/wp-content/uploads/2022/11/neozhidannaya-vstrecha-270x380.jpg"/><span>Неожиданная встреча</span></a>
<em>Bu Qi Er Zhi</em><table class="table-hom">
<tbody class="tbody-hom">
<tr><td class="naz">Сериал:</td><td>40 серий</td></tr><tr><td class="naz">Страна:</td><td>Китай</td></tr><tr><td class="naz">Год:</td><td>2022</td></tr><tr><td class="naz">Жанр:</td><td>Мелодрама, Триллер</td></tr><tr><td class="naz"><i>Добавлена:</i></td><td> 17 серия - субтитры</td></tr></tbody>
</table>
<div class="status">Выходит</div></div>

In [18]:
film.a['href']

'https://doramy.club/34015-neozhidannaya-vstrecha.html'

In [21]:
film.a.span.text

'Неожиданная встреча'

In [23]:
film.a.img['src']

'https://doramy.club/wp-content/uploads/2022/11/neozhidannaya-vstrecha-270x380.jpg'

# Сбор всех ссылок на фильмы

Постраничное сканирование, собираем все ссылки на отдельные фильмы

In [30]:
film_urls = []
for page_num in range(1,480):
    while True:
        page = requests.get(URL_DORAMY+str(page_num))
        print(page.status_code)
        if page.status_code == 200:
            break
        time.sleep(2)
    soup = BeautifulSoup(page.text)
    page_films=soup.find_all("div", class_="post-home")
    for film in page_films:
        film_urls.append(film.a['href'])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [32]:
len(film_urls)

4790

In [33]:
import pandas as pd

In [36]:
df = pd.DataFrame(film_urls, columns =['url'])
df.info()

In [39]:
df.to_csv('doramy.csv', sep = ";", encoding='utf-8')

# Тестирование на скрабинг страницы

In [ ]:
df = pd.read_csv('doramy.csv', sep=';')
df

In [46]:
df.iloc[0].url

'https://doramy.club/34127-policejskij-uchastok-ryadom-s-pozharnoj-chastyu.html'

In [48]:
r = requests.get(df.iloc[0].url)
film=BeautifulSoup(r.text)
film.article.h1.text

In [69]:
film.article.select('div.poster img')[0]['src']

'https://doramy.club/wp-content/uploads/2022/11/policejskij-uchastok-ryadom-s-pozharnoj-chastyu.jpg'

In [72]:
film.article.select('div.annotaciya')[0].text

'\nРядом с пожарной станцией расположен полицейский участок, и когда поблизости случается что-то серьезное, сотрудники сразу двух служб пересекаются, чтобы восстановить порядок в городе.\nС полицейской стороны это импульсивный детектив Чин Хо Гэ. Он часто нарушает правила и действует необдуманно, но еще ни разу не дал преступнику и шанса сбежать. Со стороны пожарной части — сотрудник Пон До Джин и медсестра Сон Соль. Они любят свою работу и всегда действуют по инструкциям. Вместе они составляют отличную команду и легко справляются со всеми чрезвычайными ситуациями.\nВыход серий в Корее по пятницам и субботам.\n'

# Парсинг основных параметров, таблицы параметров, комментариев

комментарии сохраняем как словарь

In [ ]:
def parse_comments(comments):
    comms = []
    for com in comments:
        comms.append(dict(author=com.select('div.ct-author')[0].text,
                          time=com.select('div.ct-ti')[0].text,
                          comment=com.select('div.ct-text')[0].text))
    return comms

In [ ]:
def parse_table(table):
    table_dict = {}
    for line in table.find_all('tr'):
        cols = line.find_all('td')
        table_dict[cols[0].text] = cols[1].text
    return table_dict

In [114]:
def parse_film(url):
    while True:
        page = requests.get(url)
        print(url, page.status_code)
        if page.status_code == 200:
            break
        time.sleep(2)
    film = BeautifulSoup(page.text)
    title = film.article.h1.text
    image = film.article.select('div.poster img')[0]['src']
    desription = film.article.select('div.annotaciya')[0].text
    params = parse_table(film.article.table)
    comments=parse_comments(film.article.select('ul.commentlist li'))
    return pd.Series([title, image, desription, params, comments], index =['title', 'image', 'desription', 'params', 'comments'])

In [115]:
parse_film('https://doramy.club/34127-policejskij-uchastok-ryadom-s-pozharnoj-chastyu.html')

https://doramy.club/34127-policejskij-uchastok-ryadom-s-pozharnoj-chastyu.html 200


title         Полицейский участок рядом с пожарной частью до...
image         https://doramy.club/wp-content/uploads/2022/11...
desription    \nРядом с пожарной станцией расположен полицей...
params        {'Просмотр:': '1 час 5 мин', 'Сериал:': '12 се...
comments      [{'author': 'Leodo', 'time': '16 ноября 2022',...
dtype: object

In [121]:
df_extend=df.apply(lambda row: parse_film(row[1]),axis=1)

https://doramy.club/34127-policejskij-uchastok-ryadom-s-pozharnoj-chastyu.html 200
https://doramy.club/11136-pogrebyonnyj-gorod-sokryt-vse-ogni.html 200
https://doramy.club/34047-lyubov-zdes-zapreshhena.html 200
https://doramy.club/33308-tysyacha-let-dlya-tebya.html 200
https://doramy.club/33385-superzvezda-2550.html 200
https://doramy.club/34015-neozhidannaya-vstrecha.html 200
https://doramy.club/31651-noch-predatelstva.html 200
https://doramy.club/34018-zavodila-i-princessa.html 200
https://doramy.club/32469-lun-zhi-i-tebe-konec-3.html 200
https://doramy.club/30869-ona-i-eyo-idealnyj-muzh.html 200
https://doramy.club/34167-kto-to.html 200
https://doramy.club/32850-lyubov-nachinaetsya-s-braka.html 200
https://doramy.club/34154-slabyj-geroj.html 200
https://doramy.club/34151-feya-s-kartinki.html 200
https://doramy.club/34138-yarkaya-luna-poklonyaetsya-tvoemu-serdcu.html 200
https://doramy.club/32501-shhepki-agarovogo-dereva.html 200
https://doramy.club/806-glava-klana-naotora.html 200


In [120]:
df_extend

,title,image,desription,params,comments
0,Полицейский участок рядом с пожарной частью до...,https://doramy.club/wp-content/uploads/2022/11...,\nРядом с пожарной станцией расположен полицей...,"{'Просмотр:': '1 час 5 мин', 'Сериал:': '12 се...","[{'author': 'Leodo', 'time': '16 ноября 2022',..."
1,Погребённый город: Сокрыть все огни дорама (2018),https://doramy.club/wp-content/uploads/2019/03...,\nГлавная героиня дорамы «Погребённый город: С...,"{'Просмотр:': '35 мин.', 'Сериал:': '36 серий'...","[{'author': 'mysize', 'time': '26 октября 2022..."


In [124]:
df_extend.to_pickle('doramy.pkl')

In [125]:
df_extend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4790 entries, 0 to 4789
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       4790 non-null   object
 1   image       4790 non-null   object
 2   desription  4790 non-null   object
 3   params      4790 non-null   object
 4   comments    4790 non-null   object
dtypes: object(5)
memory usage: 187.2+ KB


In [126]:
df_extend.head()

,title,image,desription,params,comments
0,Полицейский участок рядом с пожарной частью до...,https://doramy.club/wp-content/uploads/2022/11...,\nРядом с пожарной станцией расположен полицей...,"{'Просмотр:': '1 час 5 мин', 'Сериал:': '12 се...","[{'author': 'Leodo', 'time': '16 ноября 2022',..."
1,Погребённый город: Сокрыть все огни дорама (2018),https://doramy.club/wp-content/uploads/2019/03...,\nГлавная героиня дорамы «Погребённый город: С...,"{'Просмотр:': '35 мин.', 'Сериал:': '36 серий'...","[{'author': 'mysize', 'time': '26 октября 2022..."
2,Любовь здесь запрещена дорама (2022),https://doramy.club/wp-content/uploads/2022/11...,"\nДевушка по имени Ши Чэн, школьница увлеченна...","{'Просмотр:': '10 мин', 'Сериал:': '22 серии',...","[{'author': 'Niky', 'time': '9 ноября 2022', '..."
3,Тысяча лет для тебя дорама (2022),https://doramy.club/wp-content/uploads/2022/09...,\nБогиня Юнь Си и и повелитель духов Лу Янь бы...,"{'Просмотр:': '42 мин', 'Сериал:': '36 серий',...","[{'author': 'Yasmin0906', 'time': '16 ноября 2..."
4,Суперзвезда 2550 дорама (2022),https://doramy.club/wp-content/uploads/2022/09...,\nКогда на телевидении стартовал новый и ожида...,"{'Просмотр:': '1 час 27 мин', 'Сериал:': '10 с...","[{'author': 'dawerivs', 'time': '20 сентября 2..."


In [127]:
df_new=pd.concat([df,df_extend], axis=1, ignore_index=True)

In [132]:
df_new[50:53]

,num,url,name,image,notes,params,comments
50,50,https://doramy.club/31691-dvadcat-chetyre-vkus...,Двадцать четыре вкуса долгой и счастливой жизн...,https://doramy.club/wp-content/uploads/2022/05...,\nДорама расскажет нам о Ся Юйбин — жестокой и...,"{'Просмотр:': '38 мин', 'Сериал:': '24 серии',...","[{'author': 'Seiko', 'time': '10 ноября 2022',..."
51,51,https://doramy.club/33770-svet-zhizni.html,Свет жизни дорама (2022),https://doramy.club/wp-content/uploads/2022/10...,\nДорама повествует нам о жизни адвоката Ло Бэ...,"{'Просмотр:': '45 мин', 'Сериал:': '40 серий',...","[{'author': 'JULIYATH', 'time': '14 ноября 202..."
52,52,https://doramy.club/31885-kobra.html,Кобра дорама (2021),https://doramy.club/wp-content/uploads/2022/06...,"\nЧаначон — владелец компании, у него есть ден...","{'Просмотр:': '1 час 25 мин', 'Сериал:': '17 с...","[{'author': 'Ala', 'time': '29 октября 2022', ..."


In [131]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4790 entries, 0 to 4789
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   num       4790 non-null   int64 
 1   url       4790 non-null   object
 2   name      4790 non-null   object
 3   image     4790 non-null   object
 4   notes     4790 non-null   object
 5   params    4790 non-null   object
 6   comments  4790 non-null   object
dtypes: int64(1), object(6)
memory usage: 262.1+ KB


In [130]:
df_new.columns=['num','url','name','image','notes','params','comments']

In [133]:
df_new.to_pickle('doramy_new.pkl')

# Выделение индекса из url

In [136]:
df_new.url.head(20)

0     https://doramy.club/34127-policejskij-uchastok...
1     https://doramy.club/11136-pogrebyonnyj-gorod-s...
2     https://doramy.club/34047-lyubov-zdes-zapreshh...
3     https://doramy.club/33308-tysyacha-let-dlya-te...
4       https://doramy.club/33385-superzvezda-2550.html
5     https://doramy.club/34015-neozhidannaya-vstrec...
6      https://doramy.club/31651-noch-predatelstva.html
7     https://doramy.club/34018-zavodila-i-princessa...
8     https://doramy.club/32469-lun-zhi-i-tebe-konec...
9     https://doramy.club/30869-ona-i-eyo-idealnyj-m...
10                https://doramy.club/34167-kto-to.html
11    https://doramy.club/32850-lyubov-nachinaetsya-...
12          https://doramy.club/34154-slabyj-geroj.html
13       https://doramy.club/34151-feya-s-kartinki.html
14    https://doramy.club/34138-yarkaya-luna-poklony...
15    https://doramy.club/32501-shhepki-agarovogo-de...
16     https://doramy.club/806-glava-klana-naotora.html
17       https://doramy.club/33914-i-snova-pocel

In [137]:
import re

In [141]:
re.search('/(\d*)-', 'https://doramy.club/32469-lun-zhi-i-tebe-konec-3.html',)[1]

'32469'

In [142]:
df_new['id']=0

In [145]:
df_new['id']=df_new['url'].apply(lambda x: re.search('/(\d*)-',x)[1])

In [146]:
df_new.head()

,num,url,name,image,notes,params,comments,id
0,0,https://doramy.club/34127-policejskij-uchastok...,Полицейский участок рядом с пожарной частью до...,https://doramy.club/wp-content/uploads/2022/11...,\nРядом с пожарной станцией расположен полицей...,"{'Просмотр:': '1 час 5 мин', 'Сериал:': '12 се...","[{'author': 'Leodo', 'time': '16 ноября 2022',...",34127
1,1,https://doramy.club/11136-pogrebyonnyj-gorod-s...,Погребённый город: Сокрыть все огни дорама (2018),https://doramy.club/wp-content/uploads/2019/03...,\nГлавная героиня дорамы «Погребённый город: С...,"{'Просмотр:': '35 мин.', 'Сериал:': '36 серий'...","[{'author': 'mysize', 'time': '26 октября 2022...",11136
2,2,https://doramy.club/34047-lyubov-zdes-zapreshh...,Любовь здесь запрещена дорама (2022),https://doramy.club/wp-content/uploads/2022/11...,"\nДевушка по имени Ши Чэн, школьница увлеченна...","{'Просмотр:': '10 мин', 'Сериал:': '22 серии',...","[{'author': 'Niky', 'time': '9 ноября 2022', '...",34047
3,3,https://doramy.club/33308-tysyacha-let-dlya-te...,Тысяча лет для тебя дорама (2022),https://doramy.club/wp-content/uploads/2022/09...,\nБогиня Юнь Си и и повелитель духов Лу Янь бы...,"{'Просмотр:': '42 мин', 'Сериал:': '36 серий',...","[{'author': 'Yasmin0906', 'time': '16 ноября 2...",33308
4,4,https://doramy.club/33385-superzvezda-2550.html,Суперзвезда 2550 дорама (2022),https://doramy.club/wp-content/uploads/2022/09...,\nКогда на телевидении стартовал новый и ожида...,"{'Просмотр:': '1 час 27 мин', 'Сериал:': '10 с...","[{'author': 'dawerivs', 'time': '20 сентября 2...",33385


проверка на дубли

In [147]:
df_new.id.nunique()

4789

In [149]:
df_new[df_new.duplicated(subset = ['id'], keep = 'first')]

,num,url,name,image,notes,params,comments,id
4350,4350,https://doramy.club/5255-novoe-puteshestvie-na...,Новое путешествие на запад 2 дорама (2016),https://doramy.club/wp-content/uploads/2018/03...,\nВо втором сезоне ток-шоу «Новое путешествие ...,"{'Просмотр:': '1 час 22 мин.', 'Сериал:': '9 с...","[{'author': 'Aika', 'time': '12 июня 2021', 'c...",5255


In [151]:
df_new.loc[df_new.id == '5255']

,num,url,name,image,notes,params,comments,id
4349,4349,https://doramy.club/5255-novoe-puteshestvie-na...,Новое путешествие на запад 2 дорама (2016),https://doramy.club/wp-content/uploads/2018/03...,\nВо втором сезоне ток-шоу «Новое путешествие ...,"{'Просмотр:': '1 час 22 мин.', 'Сериал:': '9 с...","[{'author': 'Aika', 'time': '12 июня 2021', 'c...",5255
4350,4350,https://doramy.club/5255-novoe-puteshestvie-na...,Новое путешествие на запад 2 дорама (2016),https://doramy.club/wp-content/uploads/2018/03...,\nВо втором сезоне ток-шоу «Новое путешествие ...,"{'Просмотр:': '1 час 22 мин.', 'Сериал:': '9 с...","[{'author': 'Aika', 'time': '12 июня 2021', 'c...",5255


In [157]:
df_new.drop([4349],inplace=True)

In [158]:
df_new.loc[df_new.id == '5255']

,num,url,name,image,notes,params,comments,id
4350,4350,https://doramy.club/5255-novoe-puteshestvie-na...,Новое путешествие на запад 2 дорама (2016),https://doramy.club/wp-content/uploads/2018/03...,\nВо втором сезоне ток-шоу «Новое путешествие ...,"{'Просмотр:': '1 час 22 мин.', 'Сериал:': '9 с...","[{'author': 'Aika', 'time': '12 июня 2021', 'c...",5255


In [160]:
df_new.set_index('id', inplace=True)

In [161]:
df_new.drop(columns=['num'], inplace=True)

In [162]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4789 entries, 34127 to 25
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   url       4789 non-null   object
 1   name      4789 non-null   object
 2   image     4789 non-null   object
 3   notes     4789 non-null   object
 4   params    4789 non-null   object
 5   comments  4789 non-null   object
dtypes: object(6)
memory usage: 261.9+ KB


In [163]:
df_new.head()

,url,name,image,notes,params,comments
id,,,,,,
34127,https://doramy.club/34127-policejskij-uchastok...,Полицейский участок рядом с пожарной частью до...,https://doramy.club/wp-content/uploads/2022/11...,\nРядом с пожарной станцией расположен полицей...,"{'Просмотр:': '1 час 5 мин', 'Сериал:': '12 се...","[{'author': 'Leodo', 'time': '16 ноября 2022',..."
11136,https://doramy.club/11136-pogrebyonnyj-gorod-s...,Погребённый город: Сокрыть все огни дорама (2018),https://doramy.club/wp-content/uploads/2019/03...,\nГлавная героиня дорамы «Погребённый город: С...,"{'Просмотр:': '35 мин.', 'Сериал:': '36 серий'...","[{'author': 'mysize', 'time': '26 октября 2022..."
34047,https://doramy.club/34047-lyubov-zdes-zapreshh...,Любовь здесь запрещена дорама (2022),https://doramy.club/wp-content/uploads/2022/11...,"\nДевушка по имени Ши Чэн, школьница увлеченна...","{'Просмотр:': '10 мин', 'Сериал:': '22 серии',...","[{'author': 'Niky', 'time': '9 ноября 2022', '..."
33308,https://doramy.club/33308-tysyacha-let-dlya-te...,Тысяча лет для тебя дорама (2022),https://doramy.club/wp-content/uploads/2022/09...,\nБогиня Юнь Си и и повелитель духов Лу Янь бы...,"{'Просмотр:': '42 мин', 'Сериал:': '36 серий',...","[{'author': 'Yasmin0906', 'time': '16 ноября 2..."
33385,https://doramy.club/33385-superzvezda-2550.html,Суперзвезда 2550 дорама (2022),https://doramy.club/wp-content/uploads/2022/09...,\nКогда на телевидении стартовал новый и ожида...,"{'Просмотр:': '1 час 27 мин', 'Сериал:': '10 с...","[{'author': 'dawerivs', 'time': '20 сентября 2..."


In [164]:
df_doramy = df_new.drop(columns=['comments'])

In [165]:
df_doramy.to_pickle('doramy_orig.pkl')

# Сохранение комментариев в отдельном файле

In [167]:
df_comments = df_new['comments']

In [168]:
df_comments.info()

<class 'pandas.core.series.Series'>
Index: 4789 entries, 34127 to 25
Series name: comments
Non-Null Count  Dtype 
--------------  ----- 
4789 non-null   object
dtypes: object(1)
memory usage: 74.8+ KB


In [169]:
df_comments.head()

id
34127    [{'author': 'Leodo', 'time': '16 ноября 2022',...
11136    [{'author': 'mysize', 'time': '26 октября 2022...
34047    [{'author': 'Niky', 'time': '9 ноября 2022', '...
33308    [{'author': 'Yasmin0906', 'time': '16 ноября 2...
33385    [{'author': 'dawerivs', 'time': '20 сентября 2...
Name: comments, dtype: object

In [170]:
df_comments.to_pickle('doramy_comments.pkl')

In [171]:
df1=pd.read_pickle('doramy_comments.pkl')

In [173]:
df1.info()

<class 'pandas.core.series.Series'>
Index: 4789 entries, 34127 to 25
Series name: comments
Non-Null Count  Dtype 
--------------  ----- 
4789 non-null   object
dtypes: object(1)
memory usage: 74.8+ KB
